In [1]:
import numpy as np
import time
import os
from VAE import VAE
import cPickle
import gzip


%matplotlib inline
from PIL import Image
import numpy as np
from numpy import *
from sklearn import preprocessing as prep
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.pyplot as plt
import sklearn.cross_validation as crval
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import fetch_mldata
from scipy.interpolate import interp1d
from sklearn.svm import LinearSVC
import pylab
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.mixture import GMM
from sklearn.neighbors import KernelDensity
from sklearn import datasets
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import svm
import csv
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

def write_answers(predictions, description):
    fp=open(description+'.txt','w')
    fp.write('Point_ID,Output\n')
    i=1
    for a in predictions:
        fp.write(str(i)+','+str(a)+'\n')
        i+=1
    fp.close()



WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
/Users/mara/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
X=[]
with open('density_data.csv','r') as f:
    reader = csv.reader(f)
    for row1 in reader:
        X.append(row1)
#print size(X)

X=np.asarray(X)
X=X[:,1:15] # removing the first column since it contains indexes

print 'Data dimensions: ' 
print X.shape

X=X.astype(np.float)


X= np.ravel(X)
X= X.reshape(5000, 14)
#print(shape(X))

# splitting .. 
selX = X # (atm) we take all data --> no splitting 

# scaling
std_scal=prep.StandardScaler()
selX_scaled=std_scal.fit_transform(selX)
print('Original mean: ' +str(mean(selX))+' and std:'+ str(std(selX)))
print('After-cleaning mean: '+str(mean(selX_scaled))+' and std: '+str(std(selX_scaled)))


X=selX_scaled


Data dimensions: 
(5000, 14)
Original mean: 0.189320842874 and std:2.26790054052
After-cleaning mean: 4.3038588566e-17 and std: 1.0


In [48]:

np.random.seed(42)

hu_encoder = 400
hu_decoder = 400
n_latent = 7
continuous = True
n_epochs = 10

if continuous:
    print "Loading dataset"
    # Retrieved from: http://deeplearning.net/data/mnist/mnist.pkl.gz
    #f = open('freyfaces.pkl', 'rb')
    #x = cPickle.load(f)
    #f.close()
    x_train = X
    x_valid = X[0:100,:]
else:
    print "Loading MNIST data"
    # Retrieved from: http://deeplearning.net/data/mnist/mnist.pkl.gz
    f = gzip.open('mnist.pkl.gz', 'rb')
    (x_train, t_train), (x_valid, t_valid), (x_test, t_test) = cPickle.load(f)
    f.close()

path = "./"



Loading dataset


In [49]:
print x_valid.shape
print x_train.shape

(100, 14)
(5000, 14)


In [50]:

print "instantiating model"
model = VAE(continuous, hu_encoder, hu_decoder, n_latent, x_train)


batch_order = np.arange(int(model.N / model.batch_size))
epoch = 0
LB_list = []

if os.path.isfile(path + "params.pkl"):
    print "Restarting from earlier saved parameters!"
    model.load_parameters(path)
    LB_list = np.load(path + "LB_list.npy")
    epoch = len(LB_list)

if __name__ == "__main__":
    print "iterating"
    while epoch < n_epochs:
        epoch += 1
        start = time.time()
        np.random.shuffle(batch_order)
        LB = 0.

        for batch in batch_order:
            batch_LB = model.update(batch, epoch)
            LB += batch_LB

        LB /= len(batch_order)

        LB_list = np.append(LB_list, LB)
        print "Epoch {0} finished. LB: {1}, time: {2}".format(epoch, LB, time.time() - start)
        np.save(path + "LB_list.npy", LB_list)
        model.save_parameters(path)

    valid_LB = model.likelihood(x_valid)
    print "LB on validation set: {0}".format(valid_LB)

instantiating model
iterating
Epoch 1 finished. LB: -19.5723436237, time: 73.3275411129
Epoch 2 finished. LB: -17.0188632784, time: 73.4030570984
Epoch 3 finished. LB: -14.4730641975, time: 118.850583076
Epoch 4 finished. LB: -12.1290712175, time: 86.8832769394
Epoch 5 finished. LB: -10.3152590685, time: 88.7744169235
Epoch 6 finished. LB: -8.36191460827, time: 84.8691821098
Epoch 7 finished. LB: -7.07385940079, time: 86.9280591011
Epoch 8 finished. LB: -6.15108646007, time: 82.2275259495
Epoch 9 finished. LB: -5.42439088199, time: 90.7636489868
Epoch 10 finished. LB: -4.86506698872, time: 96.4873869419
LB on validation set: -4.31701334309


In [51]:
print x_train[1,:].shape

(14,)


In [58]:
l=np.zeros(3)
for i in range(3):
    l[i]=model.likelihood(x_train[i,:].reshape(1,14))

In [59]:
print l

[-4.05985663 -2.5445971  -3.54184532]


In [60]:
l=np.zeros(5000)
for i in range(5000):
    l[i]=model.likelihood(x_train[i,:].reshape(1,14))

In [61]:
print l

[-1.35839879 -5.44899808 -0.27007366 ..., -6.64637241 -4.89164263
 -4.29061996]


In [62]:
write_answers(l, 'variational_autoencoder_3rd_attempt')